In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint as pp

In [ ]:
df = pd.read_csv('../input/drug-classification/drug200.csv',delimiter=',')


# Exploratory Research

In [ ]:
df.sample(10)

In [ ]:
print(df.isnull().sum())


In [ ]:
print(df.describe(include='all'))


In [ ]:
print(df.shape)

In [ ]:
types_lookup = {}
types_lookup['BP'] = list(df['BP'].unique())
types_lookup['Cholesterol'] = list(df['Cholesterol'].unique())
types_lookup['Drug'] = list(df['Drug'].unique())
types_lookup['Sex'] = list(df['Sex'].unique()) 
pp(types_lookup)

In [ ]:
sns.countplot(x=df.Drug,data=df)

# At first wanted to check if there is imbalance in target column because of the small dataset. No big difference btw these categories.
> 

In [ ]:
df.groupby('Drug')['BP'].value_counts().unstack().plot.bar()
df.groupby('Drug')['Cholesterol'].value_counts().unstack().plot.bar()
df.groupby('Drug')['Sex'].value_counts().unstack().plot.bar()

I think it is a good shot and we have gained new insight. We can easily see that there is nobody with normal or low cholesterol levels among patients with drug-a or drug-b.Also all of patients with drug-c have low bloodpressure and high cholesterol. And we have to remember the counts of observations in each drug category (ps: drug-c is least-frequency element.Maybe we can deep our exploration about drug-c using bivariate relationship analyze with other columns (Age,SodiumPotassium ratio). 
It would be wrong if we didn't check if there is any gender-specific drugs or not (Gender-specific drugs are those exclusively indicated for either male or female patients).

In [ ]:
df.boxplot(column=['Age'], by=['Drug'], figsize=(12, 8))


We can see there is possible age threshold for using drug-B and drug-A. Other than that we can guess that there is no age limit for Drug-(Y,C,X).

In [ ]:
# I just wanted to see threshold valus for drug-a and drug-b
gb = df.groupby('Drug')
print(f"For drug-B : {gb.get_group('drugB')['Age'].describe()}")  # drug b segment has minimum age of 51. 
print(f"For drug-A : {gb.get_group('drugA')['Age'].describe()}")  # drug a segment has max age of 50. 

Drug vs Sodium-Potassium Ratio Using Swarmplot

In [ ]:
plt.figure(figsize=(12, 8))
sns.swarmplot(x='Drug',
              y='Na_to_K',
              data=df)

plt.title('Drug and Sodium/Potassium Ratio')


Clearly DrugY has dependency with Sodium-potassium ratio.We can say that because; 
There is nobody has ratio lower than around 15,
and ratio for other patients using other drug types does not exceed around 15.

The higher the sodium-potassium ratio, the greater the chances of dying from cardiovascular disease, a heart attack, or for any reason at all (Archives of Internal Medicine, July 11, 2011).I just added this information because i believe that we have to search for the terms we are not familiar with.Maybe next time we can look for some correlation between the NA-K ratio with BP ;) 

In [ ]:
df_before_preprocess = df.copy()

# Preprocess , Decision Tree Learning Model Creation,  Fitting and Results 

As we saw in Exploratory Research part, some filtering might be play important role for explaining underlying function of data.

In [ ]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cols_ = ['Sex', 'BP', 'Cholesterol', 'Drug']
for col in cols_:
    df[col] = label_encoder.fit_transform(df[col])

In [ ]:
from sklearn.model_selection import train_test_split
X = df.drop('Drug', axis=1)
y = df['Drug']
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.2, stratify = y) 

Train_test_split functions stratify argument in might be important for having a balanced number of examples for each class label. 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()


In [ ]:
dt.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix
predictions = dt.predict(X_test)
print(accuracy_score(y_test,predictions))

In [ ]:
print(df_before_preprocess['Drug'])
print(df['Drug'])

In [ ]:
from sklearn import tree
feature_names = list(X_train.columns)
class_names = list(types_lookup['Drug'])

fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (4,4), dpi=300)
tree.plot_tree(dt,
               feature_names = feature_names, 
               class_names=class_names,
               filled = True);
